In [ ]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
import keras_tuner

# Function to load and preprocess images
def loading_preprossing(path):
    img_list = []
    for img_path in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (255, 255))
        img = img / 255.0
        img_list.append(img)
    return img_list

# Load the datasets
normal_images = loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL')
pneumonia_images = loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA')

# Randomly sample the specified number of images from each class
normal_samples = random.sample(normal_images, 1340)
pneumonia_samples = random.sample(pneumonia_images, 2000)

# Combine the datasets
train_images = normal_samples + pneumonia_samples
train_labels = [0] * len(normal_samples) + [1] * len(pneumonia_samples)

# Convert to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Convert labels to categorical (one-hot encoding)
num_classes=2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [ ]:
def build_model(hp):
    model=models.Sequential()
    model.add(layers.Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(255,255,3)))
    model.add(layers.MaxPool2D(pool_size=(3,3)))
    model.add(layers.Dropout(rate=.25))
    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(x_train.shape[1],)))
    model.add(layers.MaxPool2D(pool_size=(3,3)))
    
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers',1,5)):
        model.add(layers.Dense(units=hp.Int(f"units{i}",min_value=32,max_value=512,step=64),
                              activation='relu'))
        model.add(layers.Dense(units=hp.Int(f"units{i}",min_value=32,max_value=512,step=32),
                              activation='relu'))
    model.add(layers.Dense(2,'softmax'))
    learning_rate=hp.Float('lr',min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"])
    return model
build_model(keras_tuner.HyperParameters())

In [ ]:
hp=keras_tuner.HyperParameters()
print(hp.Int("units",min_value=32,max_value=512,step=32))

In [ ]:
tuner=keras_tuner.RandomSearch(hypermodel=build_model,
                              objective='val_accuracy',
                              max_trials=3,
                              executions_per_trial=2
                              )

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
models=tuner.get_best_models(num_models=2)
best_model=models[0]
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
from tensorflow.keras import models
from kerastuner import HyperParameters

best_hp = tuner.get_best_hyperparameters(5) [0]

best_hp.values



In [ ]:
model = tuner.hypermodel.build(best_hp)


In [ ]:
model.fit(x=x_train,y=y_train,epochs=10)

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


In [ ]:
# Evaluate the loaded model on test data
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')
print(f'\nTest loss: {test_loss}')

In [ ]:
test_class1=loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/test/NORMAL')
test_class2=loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA')

In [ ]:
len(test_class1)

In [ ]:
test_list=test_class1+test_class2
test_label=[0]*len(test_class1)+[1]*len(test_class2)
print('the label lenght: ',len(test_label))
print('the train list: ',len(test_list))

In [ ]:
test_label

In [ ]:
test_list=np.array(test_list)
Preds=model.predict(test_list)


In [ ]:
preds=np.argmax(np.round(Preds),axis=1)
preds

In [ ]:
import pandas as pd
df=pd.DataFrame({'real':test_label,'predicted':preds})

In [ ]:
df

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(test_label,preds)
print(f'Accuracy: {accuracy}')

# Generate a classification report
class_report = classification_report(test_label,preds, target_names=['Normal', 'Pneumonia'])
print('Classification Report:\n', class_report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(test_label,preds)
print('Confusion Matrix:\n', conf_matrix)

In [ ]:
model.save('final exp.h5')
print('save')